<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #3: Upload clips to Zooniverse</h1>
<h3 align="right">Written by @jannesgg and @vykanton</h3>
<h5 align="right">Last updated: Mar 22, 2022</h5>

# Set up and requirements

### Import Python packages

In [1]:
# Set the directory of the libraries
import sys
sys.path.append('..')

# Set to display dataframes as interactive tables
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t3_utils as t3
import kso_utils.project_utils as p_utils

print("Packages loaded successfully")

<IPython.core.display.Javascript object>

Packages loaded successfully


### Choose your project

In [2]:
project_name = t_utils.choose_project()

Dropdown(description='Project:', options=('Koster_Seafloor_Obs', 'Spyfish_Aotearoa', 'SGU'), value='Koster_Sea…

In [3]:
project = p_utils.find_project(project_name=project_name.value)

### Initiate sql and zoo project

In [4]:
# Initiate db
db_info_dict = t_utils.initiate_db(project)

Enter your username for SNIC server········
Enter your password for SNIC server········
Updated sites
Updated movies
Updated species


In [ ]:
# Connect to Zooniverse project
zoo_project = t_utils.connect_zoo_project(project)

# Retrieve info about zooniverse clips

In [ ]:
zoo_info_dict = t_utils.retrieve__populate_zoo_info(project = project, 
                                                    db_info_dict = db_info_dict,
                                                    zoo_project = zoo_project,
                                                    zoo_info = ["subjects"])

# Retrieve info about movies hosted in the server

In [ ]:
# Store info about the movies available in the server
available_movies_df = t3.retrieve_movie_info_from_server(project = project,
                                                         db_info_dict = db_info_dict)

### Select the movie you want to upload to Zooniverse

In [ ]:
movie_i = t3.movie_to_upload(available_movies_df = available_movies_df)

### Check if movie is already in Zooniverse

Remember to query the newest zooniverse data to get the most up to date list of clips uploaded

In [ ]:
# Check movie hasn't been uploaded to Zooniverse
t3.check_movie_uploaded(movie_i = movie_i.value,
                        db_info_dict = db_info_dict)

### Specify the number of clips and clip length

In [ ]:
clip_selection = t3.select_clip_n_len(movie_i = movie_i.value,
                                      db_info_dict = db_info_dict)

### Review the clips that will be created

In [ ]:
t3.review_clip_selection(clip_selection = clip_selection, 
                         movie_i = movie_i.value)

### Create the clips

In [ ]:
clips_to_upload_df = t3.create_clips(available_movies_df = available_movies_df, 
                                     movie_i = movie_i.value,
                                     db_info_dict = db_info_dict,
                                     clip_selection = clip_selection,  
                                     project = project)

Make sure your workflows in Zooniverse have different names to avoid issues while selecting the workflow id

### Review clips

In [ ]:
# Review the size of the modified clips
t3.check_clip_size(clips_df = clips_to_upload_df)

### Video modifications

In [ ]:
# Select the video modification
clip_modification = t3.select_modification()

In [ ]:
# Modify the clips
clips_ready_df = t3.modify_clips(clips_to_upload_df = clips_to_upload_df.reset_index(drop=True), 
                                 movie_i = movie_i.value,
                                 clip_modification = clip_modification.label, 
                                 modification_details = clip_modification.value,
                                 project = project)

In [ ]:
# Review the size of the modified clips
t3.check_clip_size(clips_df = clips_ready_df)

In [ ]:
# Compare the original and modified clips
t3.compare_clips(df = clips_ready_df)

### Set Zooniverse metadata

In [ ]:
upload_to_zoo, sitename, created_on = t3.set_zoo_metadata(db_info_dict = db_info_dict, 
                                                          df = clips_ready_df,
                                                          project = project)

### Upload clips to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

In [ ]:
t3.upload_clips_to_zooniverse(upload_to_zoo = upload_to_zoo, 
                              sitename = sitename,
                              created_on = created_on,
                              project = project.Zooniverse_number)

In [ ]:
#END